In [47]:
import pickle
import numpy as np
import pandas as pd
import spacy
import fastBPE
import re
from collections import Counter, defaultdict

In [48]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator
from sklearn.model_selection import train_test_split
# device = torch.device("cuda")


In [50]:
!pip3 install indic-nlp-library

In [51]:
# from indicnlp.tokenize import indic_tokenize
# df = pd.read_pickle("./en_hi.pkl")
# en_tokenizer = spacy.load('en')

from indicnlp.tokenize import indic_tokenize
# df = pd.read_pickle("./en_hi.pkl")
df = pd.read_csv('./train.txt', delimiter = "\t", header = None)
df.rename(columns = {0 : 'english', 1 : 'hindi'}, inplace = True)
en_tokenizer = spacy.load('en_core_web_sm')
df


,english,hindi
0,Batman vs Superman,batman vs superman
1,"The director is Zack Snyder, 27% Rotten Tomato...","Zack Snyder director hai, 27% Rotten Tomatoes,..."
2,Not very popular it seems,lagta hai bahut popular nahi hai
3,But the audiences liked it. It has a B cinema ...,"but audience ne like kiya, iska cinema score B..."
4,Yes,yes
...,...,...
8055,Ok.,Thik hai
8056,ok bro,Thik hai bhai
8057,shall we continue?,Kya ham chalu kar sakte hai?
8058,do you like we can,Kya aapko pasand hai hamare saath


In [54]:

def build_vocab(corpus: str) -> dict:
    """Step 1. Build vocab from text corpus"""

    # Separate each char in word by space and add mark end of token
    tokens = [" ".join(word) + " </w>" for word in corpus.split()]
    
    # Count frequency of tokens in corpus
    vocab = Counter(tokens)  

    return vocab


def get_stats(vocab: dict) -> dict:
    """Step 2. Get counts of pairs of consecutive symbols"""

    pairs = defaultdict(int)
    for word, frequency in vocab.items():
        symbols = word.split()

        # Counting up occurrences of pairs
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += frequency

    return pairs


def merge_vocab(pair: tuple, v_in: dict) -> dict:
    """Step 3. Merge all occurrences of the most frequent pair"""
    
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    final_p.append([p,pair])
    for word in v_in:
        # replace most frequent pair in all vocabulary
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]

    return v_out




In [56]:
final_p = []
vocab = build_vocab(" ".join(df['english']))  # Step 1

num_merges = 300  # Hyperparameter
for i in range(num_merges):

    pairs = get_stats(vocab)  # Step 2

    if not pairs:
        break

    # step 3
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
for i in range(len(df['english'])):
    sentence = df['english'][i]
    tokens = [" ".join(word) + " </w>" for word in sentence.split()]
    for j in range(len(tokens)):
        for k in final_p:
            temp = k[0].sub(''.join(k[1]), tokens[j])
            tokens[j] = temp
    df['english'][i] = " ".join(tokens)
            

In [58]:
final_p = []
vocab = build_vocab(" ".join(df['hindi']))  # Step 1

num_merges = 300  # Hyperparameter
for i in range(num_merges):

    pairs = get_stats(vocab)  # Step 2

    if not pairs:
        break

    # step 3
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
for i in range(len(df['hindi'])):
    sentence = df['hindi'][i]
    tokens = [" ".join(word) + " </w>" for word in sentence.split()]
    for j in range(len(tokens)):
        for k in final_p:
            temp = k[0].sub(''.join(k[1]), tokens[j])
            tokens[j] = temp
    df['hindi'][i] = " ".join(tokens)

In [60]:
df

,english,hindi
0,B at man</w> v s</w> S u per man</w>,ba t man</w> v s</w> su per man</w>
1,The</w> di rec t or</w> is</w> Z ac k</w> S n ...,Z ac k</w> S n y de r</w> di re c to r</w> hai...
2,N ot</w> very</w> p op ul ar</w> it</w> se em ...,lagta</w> hai</w> bahut</w> po p u l ar</w> na...
3,B u t</w> the</w> a u di en c es</w> li ked</w...,b ut</w> au di en ce</w> ne</w> li ke</w> ki y...
4,Y es</w>,ye s</w>
...,...,...
8055,O k .</w>,T hi k</w> hai</w>
8056,o k</w> b r o</w>,T hi k</w> hai</w> b hai</w>
8057,s ha ll</w> w e</w> con t in u e?</w>,Kya</w> ha m</w> cha l u </w> kar</w> sak te</...
8058,do</w> you</w> like</w> w e</w> c an</w>,Kya</w> aap ko</w> pasand</w> hai</w> ha m are...


In [61]:
# train_temp, test_data_sent = train_test_split(df, test_size = 0.2)
# train_data_sent, validation_data_sent = train_test_split(train_temp, test_size=0.125)

train_data_sent, test_data_sent = train_test_split(df, test_size = 0.2)
validation_data_sent = pd.read_csv('./dev.txt', delimiter = "\t", header = None)
validation_data_sent.rename(columns = {0 : 'english', 1 : 'hindi'}, inplace = True)
validation_data_sent

,english,hindi
0,hello,hello
1,"hello there, I have not seen this movie so im ...","hello yar, mein is movie ko nahi dekha hoon th..."
2,Alright that is fine. What is the movie?,acha tho is movie kis baare me hein?
3,The movie is The Social Network,is movie tho social network ke bare mein hein
4,I have not seen that one either.,mein aise kuch nahi dekha hoon
...,...,...
937,Yep. Thanks for chatting,Yep. Thanks baat karne ke liye
938,"thanks, I will watch it. SOunds good","Thanks, mei dekhati hui. Achi baat hai"
939,Hi!,Hi!
940,Did you like the movie Despicable Me?,kya tumhe movie Despicable Me pasand hai?


In [64]:
train_data_sent['hindi'] 

4303    ya d i</w> aap</w> a i ti ha a si k</w> ph il ...
3807                                              o k</w>
1088    C D A</w> tho</w> is</w> movie</w> ke</w> liye...
2807    L e on ar d o</w> is ke</w> liye</w> O sc ar</...
7271    a bhi</w> mujhe</w> ach ch i</w> tarah</w> se<...
                              ...                        
2321    haan .</w> yeh</w> tho</w> a bhi</w> bhi</w> c...
4651    ja is i</w> ki</w> main</w> p lo t</w> k is ke...
3583    Mujhe</w> ac tion</w> f il m s</w> pasand</w> ...
908        Kya</w> ye</w> la m b i</w> movie</w> hai?</w>
769     N o pe ,</w> par</w> me</w> cha h ta</w> h un ...
Name: hindi, Length: 6448, dtype: object

In [65]:
train_data_sent.to_json('train_data.json', orient='records', lines=True)
validation_data_sent.to_json('validation_data.json', orient='records', lines=True)
test_data_sent.to_json('test_data.json', orient='records', lines=True)

In [70]:
def hindi_tokenizer(sentence):
#     print("This is Sentence - \n",sentence)
    a = [word.text for word in en_tokenizer.tokenizer(sentence.strip().split("-")[-1].strip())]
    
#     print(a)
    return a
        
#     return [word for word in indic_tokenize.trivial_tokenize(sentence.strip().split("-")[-1].strip())]

In [71]:
def english_tokenizer(sentence):
#     a = [word.text for word in en_tokenizer.tokenizer(sentence.strip().split("-")[-1].strip())][::-1]
    a = [word.text for word in en_tokenizer.tokenizer(sentence.strip().split("-")[-1].strip())]
#     print("This is english sentence \n", sentence)
#     print(a)
    return a 

In [89]:
def english_tokenizer_subword(sentence):
    
    a = sentence.split()
    return a

In [90]:
def hindi_tokenizer_subword(sentence):
#     print("This is Sentence - \n",sentence)
    a = sentence.split()
    return a
    

In [91]:
# HINDI = Field(tokenize = hindi_tokenizer, init_token = '<sos>', eos_token = '<eos>')
# ENGLISH = Field(tokenize = english_tokenizer, init_token = '<sos>', eos_token = '<eos>', lower = True)
# fields = {'english': ('english', ENGLISH), 'hindi': ('hindi', HINDI)}
HINDI = Field(tokenize = hindi_tokenizer_subword , init_token = '<sos>', eos_token = '<eos>' , lower = True)
ENGLISH = Field(tokenize = english_tokenizer_subword, init_token = '<sos>', eos_token = '<eos>', lower = True)
fields = {'english': ('english', ENGLISH), 'hindi': ('hindi', HINDI)}

In [92]:
train_data, validation_data, test_data = data.TabularDataset.splits(path = '',
                                        train = 'train_data.json',
                                        validation = 'validation_data.json',
                                        test = 'test_data.json',
                                        format = 'json',
                                        fields = fields)

# print(len(train_data), len(validation_data), len(test_data))
train_data_iterator, validation_data_iterator, test_data_iterator = BucketIterator.splits((train_data, validation_data, test_data), batch_size = 64, device = device, sort = False)


In [93]:
ENGLISH.build_vocab(train_data, min_freq=2)
HINDI.build_vocab(train_data, min_freq=2)
print(f"English: {len(ENGLISH.vocab)}, Hindi: {len(HINDI.vocab)}")

English: 355, Hindi: 344


In [94]:
test_data.examples[2]

In [95]:
vars(test_data.examples[2])

{'english': ['so</w>', 'w', 'ha', "t's</w>", 'ne', 'x', 't</w>'],
 'hindi': ['a', 'b</w>', 'aur</w>', 'kya</w>', 'hai</w>']}

In [96]:
def evaluate(model, data_iterator, criterion):
    total_loss = 0
    model.eval()
    with torch.no_grad():
        for batch in data_iterator:
            actual = batch.hindi
            input = batch.english
            predictions = model(input, actual)
            vocab_size = predictions.shape[-1]
            predictions = predictions[1:].view(-1, vocab_size)
            actual = actual[1:].view(-1) # flattening the sentence_length x batch_size dimensions because cross entropy loss needs 1d shape and also removing the sos token
            total_loss += criterion(predictions, actual).item()
    return total_loss / len(data_iterator)

In [97]:
def train(model, train_data_iterator, optimizer, criterion):
    total_loss = 0
    model.train()    
    for batch in train_data_iterator:
        actual = batch.hindi
        input = batch.english
        optimizer.zero_grad()
        predictions = model(input, actual)
        vocab_size = predictions.shape[-1]
        predictions = predictions[1:].view(-1, vocab_size)
        actual = actual[1:].view(-1) # flattening the sentence_length x batch_size dimensions because cross entropy loss needs 1d shape and also removing the sos token
        loss = criterion(predictions, actual)
        loss.backward()   
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
    return total_loss / len(train_data_iterator)

In [98]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, p,n):
        super().__init__()
        self.n=n
        self.word_embeddings = nn.Embedding(input_size, embedding_size) 
        self.lstm = nn.LSTM(embedding_size, hidden_size)
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        
    def forward(self, input):
        # embeddings = self.dropout(self.word_embeddings(input)) # shape will be sentence_length x batch_size x embedding_size
        o, (h,c) = self.lstm(self.dropout(self.word_embeddings(input))) # output shape will be sentence_length x batch_size x hidden size, hidden shape will be 1 x batch_size x hidden_size because we are returning only the final hidden layer
        return h,c

In [99]:
class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, p,n):
        super().__init__()
        self.n=n
        self.dropout = nn.Dropout(p)
        self.word_embeddings = nn.Embedding(output_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size + hidden_size, hidden_size)
        self.linear = nn.Linear(embedding_size + hidden_size + hidden_size, output_size)
        self.output_size = output_size
        self.hidden_size = hidden_size

    def forward(self, input, h,c, z):        
        embeddings = self.dropout(self.word_embeddings(input)) # shape will be 1 x batch_size x embedding_size
        o, (h,c) = self.lstm(torch.cat((embeddings, z), dim = 2), (h,c)) # shape will be 1 x batch_size x hidden_size
        predictions = self.linear(torch.cat((embeddings, h, z), dim = 2).squeeze(0)) # both shapes will be batch_size x output_vocabulary_size
        return predictions, h, c

In [100]:
class Model(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.device = device
        self.encoder = encoder
        self.decoder = decoder
         
    def forward(self, input, actual):
        h, c = self.encoder(input)
        z = h
        input = actual[0, :]
        predictions = torch.zeros(actual.shape[0], actual.shape[1], self.decoder.output_size).to(self.device)
        for t in range(1, actual.shape[0]):
            o, h, c = self.decoder(input.unsqueeze(0), h, c, z)
            predictions[t] = o
            predicted = o.argmax(1) 
            input = predicted

        return predictions

In [101]:
EMB_DIM = 350
DIM = 512
DROPOUT = 0.5
layer=2
enc = Encoder(len(ENGLISH.vocab), EMB_DIM, DIM, DROPOUT,layer)
dec = Decoder(len(HINDI.vocab), EMB_DIM, DIM, DROPOUT,layer)

model = Model(enc, dec, device).to(device)
for name, param in model.named_parameters():
    nn.init.uniform_(param.data, -0.08, 0.08)
model

Model(
  (encoder): Encoder(
    (word_embeddings): Embedding(355, 350)
    (lstm): LSTM(350, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (word_embeddings): Embedding(344, 350)
    (lstm): LSTM(862, 512)
    (linear): Linear(in_features=1374, out_features=344, bias=True)
  )
)

In [102]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = HINDI.vocab.stoi[HINDI.pad_token])
training_losses = []
validation_losses = []

In [103]:
N_EPOCHS = 20
best_valid_loss = 10000000000
for epoch in range(0,N_EPOCHS):
    
    train_loss = train(model, train_data_iterator, optimizer, criterion)
    valid_loss = evaluate(model, validation_data_iterator, criterion)
    training_losses.append(np.exp(train_loss))
    validation_losses.append(np.exp(valid_loss))
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'encoder_decoder_subword.pt')
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')

Epoch: 01
	Train Loss: 5.289 | Train PPL: 198.055
	 Val. Loss: 8.486 |  Val. PPL: 4844.964
Epoch: 02
	Train Loss: 5.242 | Train PPL: 189.023
	 Val. Loss: 8.294 |  Val. PPL: 4001.621
Epoch: 03
	Train Loss: 5.234 | Train PPL: 187.538
	 Val. Loss: 8.329 |  Val. PPL: 4142.161
Epoch: 04
	Train Loss: 5.230 | Train PPL: 186.766
	 Val. Loss: 8.341 |  Val. PPL: 4192.256
Epoch: 05
	Train Loss: 5.228 | Train PPL: 186.484
	 Val. Loss: 8.493 |  Val. PPL: 4878.852
Epoch: 06
	Train Loss: 5.226 | Train PPL: 186.005
	 Val. Loss: 8.558 |  Val. PPL: 5209.070
Epoch: 07
	Train Loss: 5.223 | Train PPL: 185.535
	 Val. Loss: 8.726 |  Val. PPL: 6160.730
Epoch: 08
	Train Loss: 5.221 | Train PPL: 185.167
	 Val. Loss: 8.562 |  Val. PPL: 5230.277
Epoch: 09
	Train Loss: 5.218 | Train PPL: 184.525
	 Val. Loss: 8.446 |  Val. PPL: 4657.462
Epoch: 10
	Train Loss: 5.288 | Train PPL: 198.037
	 Val. Loss: 8.633 |  Val. PPL: 5611.477


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('encoder_decoder_subword.pt'))

test_loss = evaluate(model, test_data_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(training_losses, label = 'Training')
plt.plot(validation_losses, label = 'Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def translate(text):
  model.eval()
  with torch.no_grad():
    tokens = english_tokenizer(text)
    tokens = [ENGLISH.init_token] + tokens + [ENGLISH.eos_token]
    src_indexes = [ENGLISH.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_tensor = src_tensor.reshape(-1,1)

    output = model(src_tensor, src_tensor)
    output_dim = output.shape[-1]
    output = output.view(-1, output_dim)
    indices = torch.argmax(output,dim=1).tolist()
    return [HINDI.vocab.itos[x] for x in indices]

print(translate("man speaking native language:"))

In [ ]:
for i, torch_sentence_pair in enumerate(test_data):
  
    eng_sentence = vars(torch_sentence_pair)["english"]
    hin_sentence = vars(torch_sentence_pair)["hindi"]


    hindi_predicted = translate(" ".join(eng_sentence))
    hindi_predicted = " ".join(list(filter(lambda x: x != '<eos>' and x!= '<unk>', hindi_predicted))[1:])

    print(eng_sentence)
    print(hindi_predicted)
    print()
    print()
    
    if i == 500:
      break


In [ ]:
import csv

translated_sentences = open('Paper1.csv','w')
csvwriter = csv.writer(translated_sentences)
csvwriter.writerow(['Input','Predicted','Actual'])  

for i, (eng_sentence, hin_sentence) in enumerate(zip(test_data_sent["english"], test_data_sent["hindi"])):

  hindi_predicted = translate(eng_sentence)
  hindi_predicted = " ".join(list(filter(lambda x: x != '<eos>', hindi_predicted))[1:]).strip()
  csvwriter.writerow([eng_sentence.strip(), hindi_predicted.strip(), hin_sentence.strip()])

  # print(eng_sentence.strip())
  # print(hindi_predicted)
  # print()
  # print()

translated_sentences.close()



In [ ]:
# import csv

# translated_sentences = open('Paper2.csv','w')
# csvwriter = csv.writer(translated_sentences)
# csvwriter.writerow(['Input','Predicted','Actual'])  

import nltk

bleu_i = []
bleu_o = []

for i, (eng_sentence, hin_sentence) in enumerate(zip(test_data_sent["english"], test_data_sent["hindi"])):

  hindi_predicted = translate(eng_sentence)
  hindi_predicted = " ".join(list(filter(lambda x: x != '<eos>', hindi_predicted))[1:]).strip()
  # csvwriter.writerow([eng_sentence.strip(), hindi_predicted.strip(), hin_sentence.strip()])

  bleu_i.append(hin_sentence.strip())
  bleu_o.append(hindi_predicted.strip())

  # print(eng_sentence.strip())
  # print(hindi_predicted)
  # print()
  # print()

# translated_sentences.close()

BLEU_scores = []

for i in range(len(bleu_i)):
  BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu(bleu_i[i], bleu_o[i], smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method7))
  
print("Average BLEU Score:", np.mean(BLEU_scores))